# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902010


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:06<01:40,  3.47s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<01:06,  2.45s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:46,  1.79s/it]

Rendering models:  23%|██▎       | 7/31 [00:07<00:30,  1.28s/it]

...

Rendering models:  84%|████████▍ | 26/31 [00:09<00:00,  6.60it/s]

Rendering models:  94%|█████████▎| 29/31 [00:10<00:00,  7.55it/s]

Rendering models:  97%|█████████▋| 30/31 [00:10<00:00,  7.02it/s]

Rendering models: 100%|██████████| 31/31 [00:10<00:00,  6.72it/s]

equidad1                               0.075685
coralmxrquez                           0.002148
not-logged-in-d254cc3304773762c2aa     0.531147
not-logged-in-20487e5808245554c355     0.000585
BeNjEsSePh                             0.000778
JessieDawn                             0.002277
ElisabethB                             0.001189
not-logged-in-f093a038165bf7620fc7     0.120718
not-logged-in-6ab0e00fa5a467f5f742     0.001851
not-logged-in-73b973978dc7c9e2e5ac     0.000900
jzeiszler                              0.001676
Planetme                               0.001104
not-logged-in-214814e52004b42491a4    57.632295
not-logged-in-214814e52004b42491a4     0.006058
acapirala                              0.000584
Lavadude                               0.001168
ShirleyL                               0.041059
TanlerP                                0.023190
TateNachampassak                       0.000855
clars915                               0.001144
not-logged-in-df3005ef78a24d2d1908     0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())